# Billionaire Dataset - Predictive Modelling

This notebook focuses on building predictive models to estimate billionaire wealth based on demographic, geographic, industry, and economic factors.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.feature_selection import RFECV

import warnings
warnings.filterwarnings('ignore')

## 1. Data Loading
Loading and exploring basic structure of data

In [2]:
# Load data
df = pd.read_csv('../data/processed/df_cleaned.csv')

# Display basic information
print(f"Dataset shape: {df.shape}")
display(df.head())

Dataset shape: (2591, 30)


,position,wealth,industry,full_name,age,country_of_residence,city_of_residence,source,citizenship,gender,...,gdp_country,g_tertiary_ed_enroll,g_primary_ed_enroll,life_expectancy,tax_revenue,tax_rate,country_pop,country_lat,country_long,continent
0,1,211000,Fashion & Retail,Bernard Arnault & family,74,France,Paris,LVMH,France,M,...,2715518274227,65.6,102.5,82.5,24.2,60.7,67059887,46.227638,2.213749,Europe
1,2,180000,Automotive,Elon Musk,51,United States,Austin,"Tesla, SpaceX",United States,M,...,21427700000000,88.2,101.8,78.5,9.6,36.6,328239523,37.090240,-95.712891,North America
2,3,114000,Technology,Jeff Bezos,59,United States,Medina,Amazon,United States,M,...,21427700000000,88.2,101.8,78.5,9.6,36.6,328239523,37.090240,-95.712891,North America
3,4,107000,Technology,Larry Ellison,78,United States,Lanai,Oracle,United States,M,...,21427700000000,88.2,101.8,78.5,9.6,36.6,328239523,37.090240,-95.712891,North America
4,5,106000,Finance & Investments,Warren Buffett,92,United States,Omaha,Berkshire Hathaway,United States,M,...,21427700000000,88.2,101.8,78.5,9.6,36.6,328239523,37.090240,-95.712891,North America


## 2. Data Preparation
Preparing data for modelling by defining features and splitting data into training and test sets

In [3]:
# Define features and target
y = df['wealth']

# Select features based on availability in the dataset
numeric_features = ['age', 'gdp_country', 'country_pop', 'cpi_country', 
                    'g_tertiary_ed_enroll', 'life_expectancy', 'tax_revenue']
categorical_features = ['gender', 'industry', 'continent']


# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(
    df[numeric_features + categorical_features],
    df['wealth'],
    test_size=0.2,
    random_state=42
)

print(f"Training set shape: {X_train.shape}")
print(f"Test set shape: {X_test.shape}")

Training set shape: (2072, 10)
Test set shape: (519, 10)
